### 개요
- 프로젝트 주제: RNN을 활용한 장르별 노래 가사 생성 시스템
- 데이터 출처: **멜론 사이트에서 장르별 50곡 가사 크롤링**
- 한국어 형태소 분석기 konlpy의 Okt 사용하여 토큰화
- LSTM 모델 적용

# 멜론 장르별 TOP 50 크롤링 하기
  - 원하는 장르를 선택하고 TOP 50을 크롤링 합니다.


# selenium 설치하기 
 - 필요한 드라이브 설치 

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install webdriver-manager
!pip install beautifulsoup4

# 멜론 TOP 50 크롤링 하기 


In [10]:
import requests
import bs4
# 멜론 장르음악
while(True):
    genre = input("음악 장르 입력(발라드, 댄스, 힙합, 알앤비, 인디, 록, 트로트, 포크) : ")
    if genre == "발라드":
        gnrCode = "GN0100"
        break
    elif genre == "댄스":
        gnrCode = "GN0200"
        break
    elif genre == "힙합":
        gnrCode = "GN0300"
        break
    elif genre == "알앤비":
        gnrCode = "GN0400"
        break
    elif genre == "인디":
        gnrCode = "GN0500"
        break
    elif genre == "록":
        gnrCode = "GN0600"
        break
    elif genre == "트로트":
        gnrCode = "GN0700"
        break
    elif genre == "포크":
        gnrCode = "GN0800"
        break
    else:
        print("장르 입력이 잘못 되었습니다. 다시 입력해주십시오.")
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
Song_req = requests.get(f'https://www.melon.com/genre/song_list.htm?gnrCode={gnrCode}', headers = header)
Song_html = Song_req.text
Song = bs4.BeautifulSoup(Song_html)
Song_botten = Song.findAll('a')
Song_elements = Song.find_all('a', class_="btn button_icons type03 song_info")
Song_list = str(Song_elements).split('</a>,')
Song_no_list = [Song_list[i].split("\'")[1] for i in range(len(Song_list))]
# -----------------------------------------------------------------------------------------------------------------
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
Song = []
for Song_no in Song_no_list:
    # 발라드 송넘버로 해당 노래 페이지로 이동
    req = requests.get(f'https://www.melon.com/song/detail.htm?songId={Song_no}', headers = header)
    # bs4로 해당 페이지의 코드 긁어오기
    Song_html = req.text
    Song_lyrics = bs4.BeautifulSoup(Song_html)
    # 긁어온 코드 중에서 ('div', class_="lyric")에 해당하는 부분 lyrics_elements에 담기
    lyrics_elements = Song_lyrics.find_all('div', class_="lyric")
    # <br/>을 기준으로 str형으로 변형한 후 나누기
    if len(lyrics_elements) == 1:
        lyrics = str(lyrics_elements[0]).split('<br/>')
    # 필요 없는 부분 제거
    lyrics[0] = lyrics[0][81:]  # <div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->\r\n\t\t\t\t\t\t\t 제거
    lyrics[-1] = lyrics[-1][:-13]   # \r\n\t\t\t\t\t</div> 제거
    while '' in lyrics:
        lyrics.remove('')   # '' 제거
    # Song 리스트 속에 리스트 형식으로 각각 노래 넣기
    Song.append(lyrics)
Song_lyrics = []
for i in range(len(Song)):
    lyrics = ' '.join(Song[i])
    Song_lyrics.append(lyrics)
Song = Song_lyrics
Song

음악 장르 입력(발라드, 댄스, 힙합, 알앤비, 인디, 록, 트로트, 포크) : 알앤비


["어떤 맘이었을까  처음으로 건넨 '사랑해' 란 말 뭘 몰랐던 그땐 왜 그랬을까 떠오르는 건 It's love 뻔한 이야기 속에 맘 졸이던 사랑을 추억하겠지만 참 아쉽게도 girl  It's not about you I heard you're talking about me Are you still thinking about we? 모두 니 얘기 같다며 헛소릴 하는 너 but I ain’t talkin' bout you Are you still wishing that I’ll be Writing about all of our memories? 이게 바로 내 유일한 처음이자 마지막 time I’ll be talkin’ bout you 네게 미련 가진 적 없어 너로 인해 울어본 적도 네 소식에 난 걱정이 돼 ‘Cause baby you’re so delusional   화려한 색깔들로 채워졌던 사진들을 추억하겠지만 참 아쉽게도 It’s still not about you (Just gotta tell you that I) I heard you're talking about me Are you still thinking about we? 모두 니 얘기 같다며 헛소릴 하는 너 but I ain’t talkin' bout you Are you still wishing that I’ll be Writing bout all of our memories? 이게 바로 내 유일한 처음이자 마지막 time I’ll be talkin’ bout you Is this what you wanted? 누구의 이야기일까? 꼭 이 노래가 너에게 닿았음 해 I want you to feel, I want you to feel your shame I'll tell you once, I'm done with us (Hey!) I heard you're talking about me (Talkin’ bout me) Are you still thinking about we? (Oh no) 모두 니 

In [8]:
아len(Song)

50

In [ ]:
corpus= []

for idx, sentence in enumerate(Song):
    if len(sentence) == 0: continue           # 길이가 0인 문장 제외
    if ('[' and ']') in sentence: continue    # 대괄호가 들어가는 문장 제외
    
    corpus.append(sentence)


print(f'원래 데이터 크기 : {len(Song)}')
print(f'공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : {len(corpus)}')

corpus = list(set(corpus))          # 중복불가한 set의 특성을 이용하여 중복되는 문장 제외 후 다시 리스트로 변환
print(f'중복되는 문장 제외한 데이터 크기 : {len(corpus)}')

원래 데이터 크기 : 47
공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : 47
중복되는 문장 제외한 데이터 크기 : 47


In [ ]:
import glob  #파일읽어오기
import re
import numpy as np
import tensorflow as tf
import bs4 
import requests

In [ ]:
# 전처리 함수 정의
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                     # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)     # 2
    sentence = re.sub(r'[" "]+', " ", sentence)             # 3
    sentence = re.sub(r"[^가-힣0-9a-zA-Z?.!,¿]+", " ", sentence)   # 4
    sentence = sentence.strip()                             # 5
    sentence = '<start> ' + sentence + ' <end>'             # 6
    return sentence

In [ ]:
corpus_tokenized = []

for sentence in corpus:
    # 데이터 전처리
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus_tokenized.append(preprocessed_sentence)
        
# 정제된 결과를 10곡 확인
corpus_tokenized[:10]

['<start> 모두 한 공간에 넘쳐 승낙은 끄덕임 속에 멈춰 내 것이 곧 모두의 것 함께 나누며 거닐던 그 벗 웃고 눈물 흘릴 때도 나눌줄 알던 간간한 태도 소리없이 손 내밀어 서로 지켜준 각자의 mirror 벌써 그리워져 짧기만 했던 그 때 이젠 꿈이라는걸 알기에 입술만 깨물 뿐 보기 힘들어져 널부러진 공감대 이젠 기억속 미소 옆에 남아 흐르는 눈물 뿐 큰 눈망울은 여전히 멈출수 없는 시간의 journey 영원한 약속은 어려워 마침표는 모두를 넘어버려 나만이 불러야 할 노래들 난 많이 즐거웠다 느꼈어 만만히 봐왔던 그리움들 만반의 준비가 모자랐어 벌써 그리워져 짧기만 했던 그 때 이젠 꿈이라는걸 알기에 입술만 깨물 뿐 보기 힘들어져 널부러진 공감대 이젠 기억속 미소 옆에 남아 흐르는 눈물 뿐 이미 익숙해져 대면대면한 관계 이젠 침묵속 굳은 표정만을 가리는 그늘 뿐 <end>',
 '<start> kyt와함께 party 난리나지 모두 다 날지 이건 미쳐 다같이 노래하고 춤을 추는 곡 마음대로 하고 싶은 날 있어요 alright alright 그 날이 바로 오늘 이잖아요 rrrrr hey 눈 감으면 항상 그려왔던 낙원을 만들어요 편한 운동화에 트레이닝복 굿모닝 커피한잔 들고 baby 잔소린 오늘만은 노코멘 no coment 내 버려둬 날 좀 run away go away 저멀리 far away 사람들 시선을 저 멀리 i don t care 누구나 hey 함께야 hey 나를 hey 따라해 hey 아침이 올때까지 all way 잊어버려 cha 답답한 도시를 떠나버려 ha 모두 뒤로 하고 rrrrr hey 오늘밤엔 신나게 춤을 춰 춤을 춰 just go 달려볼까 cha 갑갑한 이 맘을 떨쳐볼까 ha 오늘만큼은 오늘만은 처음부터 처음부터 새롭게 시작해 다시 한번 시작해 빽가 간다 시원하게 부서지는 바다위에 파도 봐도봐도 아름다운 나는 너의 바보 오늘도 내일도 매일똑 같은 일상의 반복 i don t care amp it s ok 난 익숙해 그렇게 내가 너를 지킬게 약

# 형태소분석기 설치

In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

### 형태소 분석기
- morphs( text )
  - 텍스트에서 형태소를 반환한다
 
- nouns( text )
  - 텍스트에서 명사를 반환한다
 
- pos( text )
  - 텍스트에서 품사 정보를 부착하여 반환한다
- sentences( text )
  - 텍스트에서 문장별로 변환한다
- Hannanum: 한나눔. KAIST Semantic Web Research Center 개발.

  - http://semanticweb.kaist.ac.kr/hannanum/

- Kkma: 꼬꼬마. 서울대학교 IDS(Intelligent Data Systems) 연구실 개발.

  - http://kkma.snu.ac.kr/

- Komoran: 코모란. Shineware에서 개발.

  - https://github.com/shin285/KOMORAN

- Mecab: 메카브. 일본어용 형태소 분석기를 한국어를 사용할 수 있도록 수정.

  - https://bitbucket.org/eunjeon/mecab-ko

- Open Korean Text: 오픈 소스 한국어 분석기. 과거 트위터 형태소 분석기.

  - https://github.com/open-korean-text/open-korean-text

In [ ]:
from konlpy.tag import Kkma, Komoran, Okt, Mecab

# kkm = Kkma()
# kom = Komoran()
# mec = Mecab()
okt = Okt()

In [ ]:
# okt 토큰화
ko_tokenized = []

for c in corpus_tokenized:
   ko_tokenized.append(okt.morphs(c)) # morphs => 텍스트에서 형태소를 반환한다

In [ ]:
# 각 곡의 시작과 끝을 <start>와 <end>로 표시
song_list = []
for song in ko_tokenized:
  song.insert(0, '<start>')
  song.append('<end>')
  song_list.append(song)

###토큰화2. 단어장 만들기 (tokenizer)

In [ ]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,         # 15000개 단어를 기억할 수 있음
        filters=' ',
        oov_token="<unk>"        # 포함되지 않는 단어는 <unk> 으로 표현
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  # 토큰 15개 초과 제외
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [ ]:
tensor, tokenizer = tokenize(song_list)

In [ ]:
# 단어장의 1~10번째 단어 확인해보기

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : 을
3 : 이
4 : you
5 : 너
6 : i
7 : 에
8 : the
9 : 가
10 : 내


In [ ]:
src_input = tensor[:, :-1] # 소스문장 생성 (마지막 단어 1개 빼고 다 가져오기)

tgt_input = tensor[:, 1:]  # 타켓문장 생성 (첫번째 단어 1개 빼고 다 가져오기)

print(src_input[0])
print(tgt_input[0])

[1329    3  242 1330   38 1331  440  441 1332  903  243   15   44   16]
[   3  242 1330   38 1331  440  441 1332  903  243   15   44   16   48]


In [ ]:
# 소스문장 인덱스 -> 단어로 변환
for idx in src_input[0]:
  print(tokenizer.index_word[idx], end=" ")

관계 이 젠 침묵 속 굳은 표정 만을 가리는 그늘 뿐 < end > 

In [ ]:
# 타켓문장 인덱스 -> 단어로 변환
for idx in tgt_input[0]:
  print(tokenizer.index_word[idx], end=" ")

이 젠 침묵 속 굳은 표정 만을 가리는 그늘 뿐 < end > <end> 

###평가 데이터셋 분리

In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,       # 데이트셋 비율
                                                          shuffle=True, 
                                                          random_state=121)     # 결과를 일정하게 보여주기위해 지정

In [ ]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (37, 14)
Target Train: (37, 14)


###모델학습

In [ ]:
# Embedding() - 단어를 밀집 벡터로 만드는 역할
#             - 인공 신경망 용어로는 임베딩 층(embedding layer)을 만드는 역할을 합니다. 
#             - Embedding()은 정수 인코딩이 된 단어들을 입력을 받아서 임베딩을 수행합니다.

from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [ ]:
vocab_size = tokenizer.num_words + 1
embedding_size = 700
hidden_size = 350
# drop_rate = 0.2
model = TextGenerator(vocab_size, embedding_size , hidden_size)

In [ ]:
from tensorflow.python.ops.nn_ops import dropout
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=200, batch_size=32)

Epoch 1/200
2/2 [==============================] - 14s 854ms/step - loss: 9.6158
Epoch 2/200
2/2 [==============================] - 2s 604ms/step - loss: 9.6041
Epoch 3/200
2/2 [==============================] - 2s 618ms/step - loss: 9.5617
Epoch 4/200
2/2 [==============================] - 2s 542ms/step - loss: 9.3393
Epoch 5/200
2/2 [==============================] - 2s 604ms/step - loss: 8.5604
Epoch 6/200
2/2 [==============================] - 2s 620ms/step - loss: 7.5430
Epoch 7/200
2/2 [==============================] - 2s 573ms/step - loss: 6.6037
Epoch 8/200
2/2 [==============================] - 2s 466ms/step - loss: 5.9477
Epoch 9/200
2/2 [==============================] - 1s 374ms/step - loss: 5.5183
Epoch 10/200
2/2 [==============================] - 1s 409ms/step - loss: 5.2979
Epoch 11/200
2/2 [==============================] - 1s 372ms/step - loss: 5.1373
Epoch 12/200
2/2 [==============================] - 1s 356ms/step - loss: 5.0832
Epoch 13/200
2/2 [==================

In [ ]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break


    generated = ""
        # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        if word_index == 0:
            continue
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 자연어 문장

In [ ]:
# 크롤링한 댄스 50곡으로 200번 학습한 결과, okt 사용, <start>, <end> 추가 

print('예시1.', generate_text(model, tokenizer, init_sentence='힘내', max_len=50))
print('예시2.', generate_text(model, tokenizer, init_sentence='슬픔', max_len=50))
print('예시3.', generate_text(model, tokenizer, init_sentence='비가 내려', max_len=50))
print('예시4.', generate_text(model, tokenizer, init_sentence='i love', max_len=50))

예시1. <unk> 도 crazy 도 와 i i hmm hmm < < < > <end> 
예시2. <unk> 도 crazy 도 와 i i hmm hmm < < < > <end> 
예시3. <unk> 내려 을 도 와 i hmm hmm hmm < < end <end> 
예시4. i love 도 와 i i i hmm hmm < < < > <end> 


# 모델링 팀의 성공적인 학습방법

### 개요
- 프로젝트 주제: RNN을 활용한 장르별 노래 가사 생성 시스템
- 데이터 출처: https://github.com/sohyeon98720/NLP/tree/master/KoGPT2-finetuning_lyrics/dataset
- 2011~2019년 발라드 곡 474개를 학습데이터로 활용함
- 한국어 형태소 분석기 konlpy의 Okt 사용하여 토큰화
- LSTM 모델 적용

In [ ]:
import glob  #파일읽어오기
import re
import numpy as np
import tensorflow as tf

In [ ]:
!pip install sentencepiece #토크나이저 인스톨

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.1 MB/s 


In [ ]:
raw_corpus = []
with open("/content/2011-2019_발라드.tsv", "r") as f:
    raw = f.read().splitlines()  #문장단위 나눔
    raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus)) # 곡의 개수
print("Examples:\n", raw_corpus[:10]) #  곡 10개 가사 출력

데이터 크기: 474
Examples:
 ['lyrics', '언젠간 이 눈물이 멈추길 언젠간 이 어둠이 걷히고 따스한 햇살이 이 눈물을 말려주길 지친 내 모습이 조금씩 지겨워지는 걸 느끼면 다 버리고 싶죠 힘들게 지켜오던 꿈을 가진 것보다는 부족한 것이 너무나도 많은 게 느껴질 때마다 다리에 힘이 풀려서 난 주저앉죠 언젠간 이 눈물이 멈추길 언젠간 이 어둠이 걷히고 따스한 햇살이 이 눈물을 말려주길 괜찮을 거라고 내 스스로를 위로하며 버티는 하루하루가 날 조금씩 두렵게 만들고 나를 믿으라고 말하면서도 믿지 못하는 나는 이제 얼마나 더 오래 버틸 수 있을 지 모르겠어요 기다리면 언젠간 오겠지 밤이 길어도 해는 뜨듯이 아픈 내 가슴도 언젠간 다 낫겠지 날 이젠 도와주길 하늘이 제발 도와주길 나 혼자서만 이겨내기가 점점 더 자신이 없어져요 언젠간 이 눈물이 멈추길 언젠간 이 어둠이 걷히고 따스한 햇살이 이 눈물을 말려주길 기다리면 언젠가 오겠지 밤이 길어도 해는 뜨듯이 이 아픈 내 가슴도 언젠간 다 낫겠지 언젠간 언젠간', '차가운 가슴이 어느새 조금씩 녹아 내렸나 봐 니가 들어왔어 그리고 나도 몰래 내 가슴을 채웠어 언제부터인가 집에 돌아오면 너를 떠올리고 있는 내 모습을 보면서 내 맘 속에 니가 있는 걸 알았어 Maybe youre the one Maybe 어쩌면 어쩌면 니가 내가 기다린 반쪽인건지 Maybe it is true 언제나 너무 가까이 있어서 몰랐었나봐 Baby Im in love with you 처음엔 몰랐어 내가 널 이렇게 떠올리게 될 줄 사랑하게 될 줄 니 맘도 제발 이런 내 마음과 같기를 Maybe youre the one Maybe 어쩌면 어쩌면 니가 내가 기다린 반쪽인건지 Maybe it is true 언제나 너무 가까이 있어서 몰랐었나봐 Baby Im in love with you 너무 늦진 않았길 이제야 깨달은 내 맘 받아주길 늦게 알았지만 이제야 알았지만 이 마음은 절대 흔들리지 않아 Maybe youre the one Maybe 

In [ ]:
# 마지막곡 가사 출력
raw_corpus[-1]

'유난히 추운 날 따스한 햇살 같던 날 그대를 처음 마주했던 밤 웃음이 새어 나오고 설레는 맘을 간직했었던 그 날의 우리 점점 서로가 당연한 듯 익숙해질 때 그때 알았어야 했어 조금 섣불렀단 걸 다시는 사랑하지 않고 이별에 아파하기 싫어 내가 싫어 떠나간 그대를 기다리는 나 아닌 척하는 게 힘들어 아직도 네가 많이 보고 싶어 네 생각에 아파오는 나 그대를 탓하며 혼자서 원망했던 날 사실 이별을 인정하기 싫었어 차가워진 네 말투를 그럴 수 있다며 난 괜찮다고 아무렇지 않다고 사랑해 아직 잊지 못해 아직도 슬퍼하는 나야 돌이킬 수 없단 걸 알지만 사랑한다고 아닌 척하는 게 힘들어 아직도 네가 많이 보고 싶어 추억에 또 아파오는 나 떠나가던 날 붙잡을 수 없었던 나를 미워하며 보낸 지난날 그대가 내게 돌아오는 날만 기다려 찢어질 듯 아팠다고 날 사랑해달라고 미안해 아직 잊지 못해 아직도 슬퍼하는 나야 돌이킬 수 없단 걸 알지만 사랑한다고 아닌 척하는 게 힘들어 아직도 네가 많이 보고 싶어 추억에 또 아파오는 나 이별에 또 울고 있는 나'

In [ ]:
# 데이터 확인

corpus= []

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue           # 길이가 0인 문장 제외
    if ('[' and ']') in sentence: continue    # 대괄호가 들어가는 문장 제외
    
    corpus.append(sentence)


print(f'원래 데이터 크기 : {len(raw_corpus)}')
print(f'공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : {len(corpus)}')

corpus = list(set(corpus))          # 중복불가한 set의 특성을 이용하여 중복되는 문장 제외 후 다시 리스트로 변환
print(f'중복되는 문장 제외한 데이터 크기 : {len(corpus)}')

원래 데이터 크기 : 474
공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : 474
중복되는 문장 제외한 데이터 크기 : 474


###전처리

In [ ]:
# 전처리 함수 정의
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                     # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)     # 2
    sentence = re.sub(r'[" "]+', " ", sentence)             # 3
    sentence = re.sub(r"[^가-힣0-9a-zA-Z?.!,¿]+", " ", sentence)   # 4
    sentence = sentence.strip()                             # 5
    # sentence = '<start> ' + sentence + ' <end>'             # 6
    return sentence

In [ ]:
corpus_tokenized = []

for sentence in corpus:
    # 데이터 전처리
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus_tokenized.append(preprocessed_sentence)
        
# 정제된 결과를 10곡 확인
corpus_tokenized[:10]

['니가 너무 예뻐서 그래 니가 너무 예뻐서 그래 나는 너만 보면 원래 나는 너한테만 더 그래 빛이나 너의 얼굴 몸매 굳이 과시하지마 뭐해 더 빠져들어가 너한테 미쳤나 모든게 완벽해 딴 남자 앞에선 차갑게 내 앞에서만 솔직하게 해 해줘 가끔 난 걱정해 넌 너무 빛나서 남자들 쳐다봐 난 다 마음에 안들어 because the way you dress and your body 남자들을 미치게 해 그래서 나까지 no need high heels 그냥 청바지와 운동화 and a little bit 액세서리 t shirt 만 입어도 you re perfect baby 이런 네가 좋아 i got some more to say 다른 남자들이 그냥 너를 볼 때 난 괜시리 불안해져 니꺼 라고 말해 나한테 매일 더 원하고 또 원해 다른 남자들이 그냥 너를 볼 때 난 괜시리 불안해서 내꺼 라고 말해 너한테 매일 너 하나만 생각해 니가 너무 예뻐서 그래 니가 너무 예뻐서 그래 나는 너만 보면 원래 나는 너한테만 더 그래 미쳤나 보면 볼수록 더 머릿속 너로 가득 채워 심장까지 번져버렸어 내 숨소리만 더 가빠져 aye girl 네가 정말 눈 부시잖아 너의 아름다운 미모 빛이 나잖아 all them other guys 지나가면 반해 버려서 나는 불안 하게 돼 네게 빠진단 말이야 네 옆에 있고 싶어 널 보는 시선 너무 많아 여기 저기 거기 전부 다 남자들이잖아 so please 두 손 깍지 끼고 빌어 that you will stay by my side 제발 나를 꽉 안아 다른 남자들이 그냥 너를 볼 때 난 괜시리 불안해져 니꺼 라고 말해 나한테 매일 더 원하고 또 원해 다른 남자들이 그냥 너를 볼 때 난 괜시리 불안해서 내꺼 라고 말해 너한테 매일 너 하나만 생각해 너무 빛이나 내곁에 있어줘 너무 빛이나 나만을 밝혀줘 빛이나 네가 너무 빛이나 누가 봐도 넌 빛이나 널 볼수록 더 빛이나 baby 니가 너무 예뻐서 그래 니가 너무 예뻐서 그래 나는 너만 보면 원래 나는 너한테만 더 그래',

### 토큰화1. 한국어 형태소 분석 (okt)

In [ ]:
# 형태소분석기 설치
# 형태소분석기 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
from konlpy.tag import Kkma, Komoran, Okt, Mecab

# kkm = Kkma()
# kom = Komoran()
# mec = Mecab()
okt = Okt()

In [ ]:
# okt 토큰화
ko_tokenized = []

for c in corpus_tokenized:
   ko_tokenized.append(okt.morphs(c))

In [ ]:
# 각 곡의 시작과 끝을 <start>와 <end>로 표시

song_list = []
for song in ko_tokenized:
  song.insert(0, '<start>')
  song.append('<end>')
  song_list.append(song)

In [ ]:
# 토큰화된 각 곡의 앞 뒤에 <start>, <end>가 잘 추가 되었는지 개수로 확인해보기

print('총 곡의 수: ',len(song_list))
print('<start>의 개수: ',sum(list(map(lambda x: x.count('<start>'), song_list))))
print('<end>의 개수: ',sum(list(map(lambda x: x.count('<end>'), song_list))))

총 곡의 수:  474
<start>의 개수:  474
<end>의 개수:  474


###토큰화2. 단어장 만들기 (tokenizer)

In [ ]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,         # 15000개 단어를 기억할 수 있음
        filters=' ',
        oov_token="<unk>"        # 포함되지 않는 단어는 <unk> 으로 표현
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  # 토큰 15개 초과 제외
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [ ]:
tensor, tokenizer = tokenize(song_list)

[[   8   18    3 ...   28  177   33]
 [ 372   11   17 ... 1243  123   33]
 [ 111  265 1248 ...   11   12   33]
 ...
 [ 348  357  330 ...  190   74   33]
 [ 695  167  533 ...   38 2265   33]
 [1102   27  314 ...  181   67   33]] <keras_preprocessing.text.Tokenizer object at 0x7f4863d318d0>


In [ ]:
# 단어장의 1~10번째 단어 확인해보기

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : 이
3 : 너
4 : 내
5 : 에
6 : 가
7 : 을
8 : 나
9 : 사랑
10 : 의


In [ ]:
src_input = tensor[:, :-1] # 소스문장 생성 (마지막 단어 1개 빼고 다 가져오기)

tgt_input = tensor[:, 1:]  # 타켓문장 생성 (첫번째 단어 1개 빼고 다 가져오기)

print(src_input[0])
print(tgt_input[0])

[  8  18   3  24 254 639   8  18   3  29 411  24  28 177]
[ 18   3  24 254 639   8  18   3  29 411  24  28 177  33]


In [ ]:
# 소스문장 인덱스 -> 단어로 변환
for idx in src_input[0]:
  print(tokenizer.index_word[idx], end=" ")

나 는 너 만 보면 원래 나 는 너 한 테 만 더 그래 

In [ ]:
# 타켓문장 인덱스 -> 단어로 변환
for idx in tgt_input[0]:
  print(tokenizer.index_word[idx], end=" ")

는 너 만 보면 원래 나 는 너 한 테 만 더 그래 <end> 

###평가 데이터셋 분리

In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,       # 데이트셋 비율
                                                          shuffle=True, 
                                                          random_state=121)     # 결과를 일정하게 보여주기위해 지정

In [ ]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (379, 14)
Target Train: (379, 14)


###모델학습

In [ ]:
# Embedding() - 단어를 밀집 벡터로 만드는 역할
#             - 인공 신경망 용어로는 임베딩 층(embedding layer)을 만드는 역할을 합니다. 
#             - Embedding()은 정수 인코딩이 된 단어들을 입력을 받아서 임베딩을 수행합니다.

from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [ ]:
vocab_size = tokenizer.num_words + 1
embedding_size = 700
hidden_size = 350
# drop_rate = 0.2
model = TextGenerator(vocab_size, embedding_size , hidden_size)

In [ ]:
from tensorflow.python.ops.nn_ops import dropout
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=200, batch_size=32)

Epoch 1/200
12/12 [==============================] - 8s 24ms/step - loss: 9.2226
Epoch 2/200
12/12 [==============================] - 0s 20ms/step - loss: 7.1783
Epoch 3/200
12/12 [==============================] - 0s 18ms/step - loss: 6.6736
Epoch 4/200
12/12 [==============================] - 0s 19ms/step - loss: 6.5038
Epoch 5/200
12/12 [==============================] - 0s 18ms/step - loss: 6.4213
Epoch 6/200
12/12 [==============================] - 0s 18ms/step - loss: 6.3929
Epoch 7/200
12/12 [==============================] - 0s 19ms/step - loss: 6.3745
Epoch 8/200
12/12 [==============================] - 0s 18ms/step - loss: 6.3607
Epoch 9/200
12/12 [==============================] - 0s 18ms/step - loss: 6.3604
Epoch 10/200
12/12 [==============================] - 0s 18ms/step - loss: 6.3499
Epoch 11/200
12/12 [==============================] - 0s 18ms/step - loss: 6.3475
Epoch 12/200
12/12 [==============================] - 0s 18ms/step - loss: 6.3500
Epoch 13/200
12/12 [=====

In [ ]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break


    generated = ""
        # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        if word_index == 0:
            continue
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 자연어 문장


### 결과 (가사 생성)

In [ ]:
# 발라드 474곡으로 200번 학습한 결과, okt 사용, <start>, <end> 추가 

print('예시1.', generate_text(model, tokenizer, init_sentence='힘내', max_len=50))
print('예시2.', generate_text(model, tokenizer, init_sentence='슬픔', max_len=50))
print('예시3.', generate_text(model, tokenizer, init_sentence='i love', max_len=50))
print('예시3.', generate_text(model, tokenizer, init_sentence='비가 내려', max_len=50))

예시1. 힘내 사랑 에 젠 하는 더 건 더 더 i 까지 you <end> 
예시2. 슬픔 상처 에 지우죠 그 널 사랑 한다 더 까지 <end> 
예시3. i love and i love time believe 뭘 over you feel first kiss <end> 
예시3. <unk> 내려 a 진작 를 그렇게 보고싶다 말 해줬으면 이렇게 끝나진 않을 걸 이 젠 늦었어 <end> 
